# Get Data 
> A notebook for getting data from official sources and unzipping them to machine readable formats

- toc: true 
- badges: false
- comments: true
- categories: [jupyter]
- author: Nirant Kasliwal and Meghana Bhange
<!-- - image: images/chart-preview.png -->

In [ ]:
# hide
# !pip install requests
# !pip install pydantic
# !pip install tqdm
# !pip install pdfminer.six

In [ ]:
!which python

In [ ]:
!python --version

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# hide_input
import json
from io import StringIO
from pathlib import Path
from typing import List, Union

import requests
from pydantic import BaseModel
from tqdm.notebook import tqdm

from pdf_parsing import pdf_to_text
from textbook import Book, Chapter

In [ ]:
Path.pdfls = lambda x: [x for x in list(x.iterdir()) if x.suffix == ".pdf"]
Path.ls = lambda x: list(x.iterdir())

### Get List of Books and Download Links

In [ ]:
# collapse-hide
sheet_name = "History"
books_list = (
    f"https://api.steinhq.com/v1/storages/5fd49704f62b6004b3eb63a3/{sheet_name}"
)
r = requests.get(books_list)

In [ ]:
# collapse-hide
ncert_history_books = [Book(**x) for x in json.loads(r.text)]

## Download and Extract all Books

In [ ]:
# collapse-show
for book in tqdm(ncert_history_books):
    book.download("../data/raw")
    book.unzip("../data/extract")

In [ ]:
single_book = ncert_history_books[0]

In [ ]:
single_book.make_chapters()

In [ ]:
single_book.chapters

In [ ]:
single_book.improve_sentence_boundries()

In [ ]:
print(single_book.chapters[0].space_formatted_text)

In [ ]:
# ncert_history_books[1].download()
# ncert_history_books[1].unzip()